In [1]:
import pandas as pd
import datetime as dt
from datetime import timedelta
import time
import dateutil
from dateutil.parser import parse
import logging
import importlib
importlib.reload(logging)

<module 'logging' from '/usr/local/anaconda3/lib/python3.6/logging/__init__.py'>

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

In [3]:
logging.info("now starting...")

10:30:08 INFO:now starting...


In [4]:
logging.info("...before reading csv file.")
# read a csv into a dataframe
df = pd.read_csv('data/IMPORT_CONTROL.txt')
logging.info("...after reading csv file.")

10:30:08 INFO:...before reading csv file.
10:30:20 INFO:...after reading csv file.


In [5]:
logging.info("...before creating new column.")
# need to make a field to uniquely identify each agency
# concatenating the SITE_ID and the AGENCY will do this
named = df.assign(uniquename = df.SITE_ID.astype(str) + '-' + df.AGENCY.astype(str))
logging.info("...after creating new column.")

# new dataframe to record new info
# interval_df = pd.DataFrame(columns=['uniquename', 'timestamp', 'interval', 'origindex'])

10:30:20 INFO:...before creating new column.
10:30:29 INFO:...after creating new column.


In [6]:
# use the groupby functions to look at events for each agency
logging.info("...before groupby.")
grouped = named.groupby('uniquename')
logging.info("...after groupby.")

10:30:29 INFO:...before groupby.
10:30:30 INFO:...after groupby.


In [7]:
logging.info("...before dict")
# make a dictionary of the group names
groups = dict(list(grouped))
logging.info("...after dict")


10:30:30 INFO:...before dict
10:30:38 INFO:...after dict


In [8]:
data_rows = []
i = 0
# for each group
for name in groups:
    # print(name)
    # get the group as a dataframe
    g = grouped.get_group(name)
    # logging.info("length of group: {}".format(len(g)))
    j = 0
    t = time.process_time()
    # then go through the dataframe row by row
    for r in g.itertuples():
        # logging.info("..on row# {}".format(r))
        # only for rows > 0
        row = {}
        if j > 0 and isinstance(r.PROCESS_END_TS, str):
            # calculate seconds between this timestamp and the timestamp from the last row
            thistime = parse(r.PROCESS_END_TS)
            lasttime = parse(last)
            secs = (thistime - lasttime).total_seconds()
            # print(r.Index, thistime, lasttime, secs)
            # interval_df = interval_df.append({'timestamp': thistime, 'uniquename': name, 'interval': secs, 'origindex': r.Index}, ignore_index=True)
            # set the dictionary with the values from this calculated interval
            row['timestamp'] = thistime
            row['uniquename'] = name
            row['interval'] = secs
            row['origindex'] = r.Index
            last = r.PROCESS_END_TS
        if j == 0 and isinstance(r.PROCESS_END_TS, str):
            last = r.PROCESS_END_TS
        j += 1
        # append the dictionary to the data_rows array
        data_rows.append(row)
    i += 1
    dur = time.process_time() - t
#    logging.info("dur: {}, num: {}, avg: {}".format(dur, len(g), dur / len(g)))
    if i > 200000:
        break
    if i == 1 or i%20 == 0:
        logging.info("agency #{} out of {}".format(i, len(groups)))

# create the dataframe from the data_rows array
interval_df = pd.DataFrame(data_rows)
# then write it out to a file
interval_df.to_csv('data/intervals.csv')

10:30:39 INFO:agency #1 out of 2361
10:30:58 INFO:agency #20 out of 2361
10:31:21 INFO:agency #40 out of 2361
10:31:57 INFO:agency #60 out of 2361
10:32:29 INFO:agency #80 out of 2361
10:33:50 INFO:agency #100 out of 2361
10:34:26 INFO:agency #120 out of 2361
10:34:40 INFO:agency #140 out of 2361
10:34:52 INFO:agency #160 out of 2361
10:34:58 INFO:agency #180 out of 2361
10:35:08 INFO:agency #200 out of 2361
10:35:16 INFO:agency #220 out of 2361
10:35:27 INFO:agency #240 out of 2361
10:35:40 INFO:agency #260 out of 2361
10:35:45 INFO:agency #280 out of 2361
10:35:58 INFO:agency #300 out of 2361
10:36:11 INFO:agency #320 out of 2361
10:36:31 INFO:agency #340 out of 2361
10:36:54 INFO:agency #360 out of 2361
10:37:14 INFO:agency #380 out of 2361
10:37:27 INFO:agency #400 out of 2361
10:37:37 INFO:agency #420 out of 2361
10:37:46 INFO:agency #440 out of 2361
10:37:55 INFO:agency #460 out of 2361
10:38:05 INFO:agency #480 out of 2361
10:38:23 INFO:agency #500 out of 2361
10:38:35 INFO:agen